In [1]:
import numpy as np
import cv2 as cv
import glob

In [ ]:
left

In [4]:
ret, corners = cv.findChessboardCorners()

error: OpenCV(4.12.0) :-1: error: (-5:Bad argument) in function 'findChessboardCorners'
> Overload resolution failed:
>  - findChessboardCorners() missing required argument 'image' (pos 1)
>  - findChessboardCorners() missing required argument 'image' (pos 1)
